# 🔄 Spotify Library Sync

This notebook downloads your Spotify library and saves it locally for offline analysis.

**What it does:**
- ✅ Fetches all your playlists (owned only)
- ✅ Fetches your Liked Songs (❤️ master playlist)
- ✅ Downloads track and artist metadata
- ✅ Saves everything to `data/` as parquet files
- ✅ Incremental updates (only fetches changes)

**Run this first!** Then use `02_analyze_library.ipynb` for analysis.

**💡 Tip:** For automated daily syncs, use `src/scripts/automation/sync.py` instead (configured via cron job). See `README.md` for details.

## 1️⃣ Setup

Install dependencies and configure credentials.

In [1]:
# Install dependencies (run once)
%pip install -q pandas spotipy pyarrow tqdm python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Setup project - this adds project root to path
# From src/notebooks/, go up 2 levels to reach project root
from pathlib import Path
from notebook_helpers import setup_project

PROJECT_ROOT = setup_project(Path("../..").resolve())

/opt/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Project root: /Users/aryamaan/Desktop/Projects/SPOTIM8/spotim8


In [3]:
# Load and verify credentials from .env file
from notebook_helpers import setup_credentials

credentials_ok = setup_credentials(PROJECT_ROOT)
if not credentials_ok:
    print("⚠️  Please set up credentials before continuing!")

⚠️  No .env file found at /Users/aryamaan/Desktop/Projects/SPOTIM8/spotim8/.env
   Create one with SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, SPOTIPY_REDIRECT_URI
⚠️  Please set up credentials before continuing!


## 2️⃣ Connect to Spotify

This will open a browser window for authentication on first run.

In [4]:
# Initialize Spotify client with caching enabled
from notebook_helpers import setup_spotify_client

sf = setup_spotify_client(PROJECT_ROOT, progress=True, cache_ttl=3600)
DATA_DIR = PROJECT_ROOT / "data"

📦 API response cache enabled: /Users/aryamaan/Desktop/Projects/SPOTIM8/src/data/.api_cache (TTL: 3600s)
✅ Connected to Spotify!
📁 Data will be saved to: /Users/aryamaan/Desktop/Projects/SPOTIM8/src/data


## 3️⃣ Sync Your Library

This fetches your playlists and tracks. First run may take a few minutes.

In [ ]:
# Sync library (incremental - only fetches changes)
stats = sf.sync(
    owned_only=True,           # Only your playlists, not followed ones
    include_liked_songs=True   # Include Liked Songs as master playlist
)

print(f"\n📊 Sync complete!")

🔄 Starting library sync...
📝 106 playlist(s) changed: ❤️ Liked Songs, AJFndsJan26, AJFndsDec25, AJFndsNov25, Anthems ...
❤️  Fetching Liked Songs (your master playlist)...


Fetching Liked Songs:  73%|███████▎  | 3900/5341 [01:05<00:25, 55.68track/s]

KeyboardInterrupt: 

Fetching Liked Songs:  73%|███████▎  | 3900/5341 [01:20<00:25, 55.68track/s]

## 4️⃣ Build Full Data Tables

Now let's build all the detailed tables (tracks, artists, etc.) from the synced data.

## 5️⃣ View Your Data


In [ ]:
# Show status summary
sf.print_status()


🔄 Syncing export data...
   📁 Checking: /Users/aryamaan/Desktop/Projects/src/data
      ✅ Found: Spotify Account Data/ (data)
      ✅ Found: Spotify Extended Streaming History/ (data)
      ✅ Found: Spotify Technical Log Information/ (data)
✅ Streaming history already synced (46,383 records)
✅ Search queries already synced (763 records)
✅ Wrapped data already synced
✅ Follow data already synced (27 records)
✅ Library snapshot already synced (5,168 records)
✅ Playback errors already synced (9 records)
✅ Playback retries already synced (2 records)
✅ Web API events already synced (3,242 records)

✅ Export data sync complete!


In [ ]:
# Fetch all data tables (uses cache if available)
print("📥 Building data tables...\n")

playlists = sf.playlists()
print(f"✅ Playlists: {len(playlists):,}")

playlist_tracks = sf.playlist_tracks()
print(f"✅ Playlist-track links: {len(playlist_tracks):,}")

tracks = sf.tracks()
print(f"✅ Unique tracks: {len(tracks):,}")

track_artists = sf.track_artists()
print(f"✅ Track-artist links: {len(track_artists):,}")

artists = sf.artists()
print(f"✅ Artists: {len(artists):,}")

# Build the wide table (everything joined)
library = sf.library_wide()
print(f"✅ Library wide table: {len(library):,} rows")

📥 Building data tables...

✅ Playlists: 560
✅ Playlist-track links: 75,210
✅ Unique tracks: 5,666
✅ Track-artist links: 9,081
✅ Artists: 2,854
✅ Library wide table: 75,961 rows


## 6️⃣ Preview Your Data

In [ ]:
# Preview playlists
print("📂 Your Playlists:")
playlists[["name", "track_count"]].head(15)


        SPOTIM8 DATA STATUS
📁 Cache directory: /Users/aryamaan/Desktop/Projects/src/data
👤 User: 31iol2qamank24owygxo7kpq533y
🕐 Last sync: 2026-01-11T10:00:46.560484+00:00

📊 Cached data:
   • Playlists: 560
   • Playlist tracks: 75,210
   • Unique tracks: 5,666
   • Track-artist links: 9,081
   • Artists: 2,854



In [ ]:
# Preview tracks
print("🎵 Sample Tracks:")
tracks[["name", "album_name", "popularity", "duration_ms"]].head(10)

📂 Your Playlists:


,name,track_count
0,❤️ Liked Songs,5268
1,OtherFinds25,281
2,AJDiscovery24,100
3,AJTop24,100
4,AJDiscovery23,100
5,AJTop23,100
6,AJDiscovery22,100
7,AJTop22,100
8,AJDiscovery21,100
9,AJTop21,100


In [ ]:
# Preview artists
print("🎤 Top Artists (by followers):")
artists.nlargest(10, "followers")[["name", "genres", "popularity", "followers"]]

🎵 Sample Tracks:


,name,album_name,popularity,duration_ms
0,Night Drive,Night Drive,64,220000
1,don't u know?,Missing in Action (The Return),57,176004
2,High No More,High No More,73,198845
3,Back On 74,Volcano,80,209482
4,Afraid To Feel,Afraid To Feel,77,177524
5,DARE (feat. Shaun Ryder & Roses Gabor),Demon Days,83,244999
6,Casio,For Ever,70,234369
7,Electric Feel,Oracular Spectacular,79,229640
8,Move Your Feet,D-D-Don't Don't Stop The Beat,72,181826
9,What I Might Do,What I Might Do (Radio Edit),53,195737


In [ ]:
# Preview artists
print("🎤 Top Artists (by followers):")
artists.nlargest(10, "followers")[["name", "genres", "popularity", "followers"]]

🎤 Top Artists (by followers):


,name,genres,popularity,followers
1144,Arijit Singh,"[hindi pop, bollywood, desi, bangla pop]",93,170571717
226,Taylor Swift,[],100,149663352
759,Ed Sheeran,[soft pop],90,124275845
95,Billie Eilish,[],93,121794604
550,The Weeknd,[],96,116875799
178,Ariana Grande,[pop],95,109014155
656,Eminem,"[rap, hip hop]",91,106545048
1070,Bad Bunny,"[reggaeton, trap latino, urbano latino, latin]",99,106389089
44,Drake,[rap],98,105877620
756,Justin Bieber,[],94,86335475


## 7️⃣ Check Saved Files

In [ ]:
# List saved library data files
print(f"📁 Library data files in {DATA_DIR}:\n")

library_files = []
for f in sorted(DATA_DIR.glob("*.parquet")):
    # Only show library data files (exclude export data files)
    if f.name not in ['streaming_history.parquet', 'search_queries.parquet', 
                       'follow_data.parquet', 'library_snapshot.parquet',
                       'playback_errors.parquet', 'playback_retries.parquet',
                       'webapi_events.parquet']:
        library_files.append(f)

if library_files:
    print("📚 Library Data (from Spotify API):")
    for f in library_files:
        size_kb = f.stat().st_size / 1024
        print(f"   {f.name:30} {size_kb:>8.1f} KB")
    print(f"\n✅ {len(library_files)} library data file(s) saved")
else:
    print("⚠️  No library data files found")
    print("   Run the sync in section 3️⃣ to download your library")

📁 Files in /Users/aryamaan/Desktop/Projects/src/data:

📚 Library Data (from API):
   artists.parquet                   215.6 KB
   library_wide.parquet             2752.5 KB
   playlist_tracks.parquet           781.0 KB
   playlists.parquet                 110.5 KB
   track_artists.parquet             234.0 KB
   tracks.parquet                    657.8 KB

📥 Export Data (from Spotify exports):
   follow_data.parquet                 2.0 KB
   library_snapshot.parquet          268.1 KB
   playback_errors.parquet            15.3 KB
   playback_retries.parquet           14.6 KB
   search_queries.parquet             21.9 KB
   streaming_history.parquet        2586.9 KB
   webapi_events.parquet             103.6 KB
   wrapped_data.json                   3.6 KB


---

## ✅ Done!

Your library is now synced and saved locally. Next steps:

1. **Analyze**: Open `02_analyze_library.ipynb` for visualizations
2. **Playlist Analysis**: Open `03_playlist_analysis.ipynb` for genre clustering
3. **Listening History**: Open `04_analyze_listening_history.ipynb` to analyze your actual listening patterns
4. **Create Playlists**: Open `05_liked_songs_monthly_playlists.ipynb` to create automated playlists
5. **Find Redundancy**: Open `06_identify_redundant_playlists.ipynb` to clean up your library

**Re-sync**: Run this notebook again anytime to fetch new changes. The data is cached, so future runs are fast!

**💡 Note**: For export data (streaming history, search queries, etc.), use the sync script: `src/scripts/automation/sync.py`